In [1]:
import math
import numpy as np
import pandas as pd
#from scipy.special import logsumexpg

### Functions that will be called in the main code

In [40]:
def fitness(init_p: list, n: int, dimension: int, upper_b: int, lower_b: int, collie: object) -> list:
    fit1 = np.zeros((n,1))
    
    for i in range(30):
        fit1[i] = collie.func15_op(init_p[i,:])
        
    masked_fit = np.ma.masked_invalid(fit1)
    
    #best fit and position
    best_fit = np.min(masked_fit)
    pos = np.where(best_fit == np.min(masked_fit))

    return fit1, best_fit, pos

In [32]:
def get_index(vet1: list, vet2: list) -> int:
    idx = [i for i in range(len(vet1))]
    for i in range(len(vet1)):
        for j in range(len(vet2)):
            if vet1[i] == vet2[j]:
                idx[i] = j
    
    return idx

In [33]:
def herding(pop: list, Vt: list, fit: list, n: int, dimension: int, acc: list, time: list) -> list:
    
    fit1 = np.sort(fit, axis=0,kind="heapsort") 
    idx = get_index(fit1, fit)
    pop1 = np.zeros((n, dimension))
    Vt1 = np.zeros((n, dimension))
    acc1 = np.zeros((n, dimension))
    time1 = np.zeros((n, dimension))

    for i in range(n):
        
        pop1[i, :] = pop[idx[i], :]
        Vt1[i, :] = Vt[idx[i], :]
        acc1[i, :] = acc[i, :]
        time1[i] = time[i]
    
    return pop1, Vt1, fit1, acc1, time1

In [34]:
def generate(n: int, dimension: int, upper_b: int, lower_b: int) -> list:
    boundary_no = 1
    pop = np.zeros((n, dimension))
    
    if boundary_no == 1:
        pop = np.random.random(size= (n, dimension)) * (upper_b - lower_b) + lower_b
    
    #Unused part of the algorithm, that's why it's commented
    #if boundary_no > 1:
    #    for i in range(1,dimension):
    #        pop = np.random.random(size = (n,1))*(upper_b-lower_b)+lower_b;
    x = np.random.random(size = (n, dimension))
    M = pop

    return M, x


In [35]:
def update(pop: list, Vt: list, time: list, acc: list, n: int, dimension: int, eye: int) -> list:
    pop1 = np.zeros((n,dimension))
    for i in range(1, n):
        for j in range(1, dimension):
            #Updating the position of dogs
            if (i <= 3):
                pop1[i][j] = Vt[i][j] * time[i] + (1/2) * acc[i][j] * (np.power(time[i], 2))
            
            #Updating position of sheep
            if (i > 3):
                if eye == 1:
                    pop1[i][j] = Vt[i][j] * time[i] - (1/2) * acc[i][j] * (np.power(time[i], 2))
                else:
                    pop1[i][j] = Vt[i][j] * time[i] + (1/2) * acc[i][j] * (np.power(time[i], 2))
            
    return pop1

In [36]:
def update_v(Vt: list, n: int, dimension: int, acc: list, time: list, pop: list, fit: list, eye: int) -> list:
    #Choosing left and right dogs
    right_dog = np.random.randint(2,3)
    if (right_dog == 2):
        left_dog = 3
    else:
        left_dog = 2

    acc1 = np.ones((n,dimension))
    time1 = np.ones((n,1))
    Vt1 = Vt
    r = len(acc)
    l = len(acc)
    acc2 = np.zeros((r, l))
    fit1 = fit[0]

    #Finding Dg value to choose which sheep to gather and which to stalk
    fit2 = (fit[2] + fit[3]) / 2
    f = 0
    tempg = [0 for i in range(n)]
    temps = [0 for i in range(n)]

    #Setting parameters for eyeing
    if eye == 1:
        if fit[right_dog] < fit[left_dog]:
            acc2[left_dog, :] = -1 * acc[left_dog, :]
            f = left_dog
        else:
            acc2[right_dog, :] = -1 * acc[right_dog, :]
            f = right_dog

    for i in range(1, n):
        for j in range(1, dimension):
            #Velocity updation of dogs
            if (i <= 3):
                Vt1[i][j] = math.sqrt(np.power(Vt[i][j], 2) + (2 * acc[i][j]) * abs(pop[i][j]))

            #Velocity updation of sheep
            if (i > 3):
                if eye == 1:
                    Vt1[i][j] = math.sqrt(np.power(Vt1[f][j], 2) + (2*acc[f][j]) * abs(pop[i][j]))

                else:
                    #Velocity updation of gathered sheep
                    if (fit1 - fit[i] > fit2 - fit[i]):
                        Vt1[i][j] = math.sqrt(np.power(Vt1[0,j], 2) + (2*acc[0,j]))
                        tempg[i] = i        
                
                    #Velocity updation of stalked sheep
                    if (fit1 - fit[i] <= fit2 - fit[i]):
                        Vt1[i][j] = math.sqrt(np.power(Vt[right_dog][j]*math.tan(np.random.randint(1,89)), 2) + (2*acc[right_dog][j]*abs(pop[right_dog][j]))) + math.sqrt(np.power(Vt[left_dog][j]*math.tan(np.random.randint(91,179)), 2) + (2*acc[left_dog][j]*abs(pop[left_dog][j])))
                        Vt1[i][j] = (Vt1[i][j]) / 2
                        temps[i] = i
    
    #Updation of time and acceleration
    for i in range(1, n):
        s = 0
        for j in range(1, dimension):
            acc[i] = abs((Vt1[i][j]) - (Vt[i][j])) / (time[i])
            s += ((Vt1[i][j]) - (Vt[i][j])) / (acc[i][j])
        time1[i] = abs(np.mean(s))

    return Vt1, acc1, time1, right_dog, left_dog, tempg, temps

In [37]:
def check(pop: list, n: int, dimension: int, upper_b: int, lower_b: int, acc: list, Vt: list, time: list) -> list:
    pop1 = pop
    acc1 = acc
    time1 = time
    Vt1 = Vt
    for i in range(n):
        for j in range(dimension):

            if (pop[i][j] >= upper_b or pop[i][j] <= lower_b or pop[i][j] == 0):
                pop1[i][j] = np.random.random() * ((upper_b - lower_b) + lower_b)
                acc1[i][j] = np.random.random()
                time1[i] = np.random.random()
    
    for i in range(n):
        for j in range(dimension):
            if (math.isnan(acc[i][j]) == 1 or acc[i][j] == 0):
                pop1[i][j] = np.random.random() * ((upper_b - lower_b) + lower_b)
                acc1[i][j] = np.random.random()
                time1[i] = np.random.random()

    for i in range(n):
        for j in range(dimension):
            if (math.isnan(Vt[i][j]) == 1 or Vt[i][j] == 0):
                pop1[i][j] = np.random.random() * ((upper_b - lower_b) + lower_b)
                acc1[i][j] = np.random.random()
                time1[i] = np.random.random()

    for i in range(n):
        for j in range(dimension):
            if (math.isnan(time1[i]) == 1 or time1[i] == 0):
                pop1[i][j] = np.random.random() * ((upper_b - lower_b) + lower_b)
                acc1[i][j] = np.random.random()
                time1[i] = np.random.random()


    return pop1, acc1, time1, Vt1

In [38]:
def Ufun(x: int, a: int, k: int, m: int) -> int:
        op = k*((x-a)^m) * (x>a) + k * ((-x-a)^m) * (x<(-a))
        
        return op

class functions:
    def func_details(self, fn):
        default = "Invalid function!!!"

        return getattr(self, 'case_'+str(fn), lambda: default)()

    
    def func1(self):
        lower_b = -100
        upper_b = 100
        dimension = 30

        return lower_b, upper_b, dimension

    
    def func2(self):
        lower_b = -10
        upper_b = 10
        dimension = 30

        return lower_b, upper_b, dimension

    
    def func3(self):
        lower_b = -100
        upper_b = 100
        dimension = 30

        return lower_b, upper_b, dimension

    
    def func4(self):
        lower_b = -100
        upper_b = 100
        dimension = 30

        return lower_b, upper_b, dimension

    
    def func5(self):
        lower_b = -30
        upper_b = 30
        dimension = 30

        return lower_b, upper_b, dimension

    
    def func6(self):
        lower_b = -100
        upper_b = 100
        dimension = 30

        return lower_b, upper_b, dimension

    
    def func7(self):
        lower_b = -1.28
        upper_b = 1.28
        dimension = 30

        return lower_b, upper_b, dimension

    
    def func8(self):
        lower_b = -500
        upper_b = 500
        dimension = 30

        return lower_b, upper_b, dimension

    
    def func9(self):
        lower_b = -32
        upper_b = 32
        dimension = 30

        return lower_b, upper_b, dimension

    
    def func10(self):
        lower_b = -50
        upper_b = 50
        dimension = 30

        return lower_b, upper_b, dimension

    
    def func11(self):
        lower_b = -50
        upper_b = 50
        dimension = 30

        return lower_b, upper_b, dimension

    
    def func12(self):
        lower_b = -65.536
        upper_b = 65.536
        dimension = 2

        return lower_b, upper_b, dimension

    
    def func13(self):
        lower_b = -5
        upper_b = 5
        dimension = 4

        return lower_b, upper_b, dimension

    
    def func14(self):
        lower_b = -2
        upper_b = 2
        dimension = 2

        return lower_b, upper_b, dimension

    
    def func15(self):
        lower_b = 0
        upper_b = 1
        dimension = 3

        return lower_b, upper_b, dimension

    def func16(self):
        lower_b = 0
        upper_b = 1
        dimension = 6

        return lower_b, upper_b, dimension

    def func17(self):
        lower_b = 0
        upper_b = 10
        dimension = 4

        return lower_b, upper_b, dimension

    def func18(self):
        lower_b = 0
        upper_b = 10
        dimension = 4

        return lower_b, upper_b, dimension

    def func19(self):
        lower_b = 0
        upper_b = 10
        dimension = 4

        return lower_b, upper_b, dimension
    
    #Working
    def func1_op(self, x):
        op = np.sum(np.power(x,2))

        return op
    #Working
    def func2_op(self, x):
        op = np.sum(np.abs(x)) + np.prod(np.abs(x))           

        return op
    #Not working
    def func3_op(self, x):
        dim = x
        op = 0
        for i in range(len(dim)):
            op += np.power(x,2)
        
        return op
    #Working
    def func4_op(self, x):
        
        op = max(abs(x))

        return op
    #Not working
    def func5_op(self, x):

        dim = x
        op = (100*x) - np.power(np.power(x,2),2) + np.power((x-1),2)

        return op
    #Working
    def func6_op(self, x):
        
        op = sum(np.power(abs(x+0.5), 2))

        return op
    #Working
    def func7_op(self, x):

        dim = x
        op = sum(x * np.power(x,4) + np.random.random())

        return op
    #Working
    def func8_op(self, x):

        op = np.sum(np.sum(-x) * math.sin(math.sqrt(np.sum(abs(x)))))

        return op
    #Not working
    def func9_op(self, x):

        dim = x
        op = np.sum(-20 * math.exp(-0.2 * math.sqrt(np.sum(abs(np.power(x,2)/dim)))) - math.exp(math.cos(2 * math.pi * np.sum(x))/dim) + 20 + math.exp(1))

        return op
    #Not working
    def func10_op(self, x):

        dim = x
        op = (math.pi/dim) * (10* (math.sin(math.pi * 1 + np.power((x + 1)/4)),2)) + np.sum((np.power(((x+1)/4),2)) * (1+10 * ((math.sin(math.pi * float(np.power(1+(x+1)/4,2)))))) + float(np.power((x+1)/4,2))) + np.sum(Ufun(x, 10, 100, 4))

        return op
    #Not working
    def func11_op(self, x):

        dim = x
        op = 0.1 * (np.power((math.sin(3*math.pi*x)), 2) + np.sum(np.power((x-1), 2) * np.power(1+(math.sin(3*math.pi*x)), 2)) + np.power((x-1), 2)* np.power(1+(math.sin(2*math.pi*x)), 2)) + np.sum(Ufun(x,5,100,4))

        return op
    #Not working
    def func12_op(self, x):

        aS = np.matrix
        bS = [0 for i in range(25)]
        
        for j in range(1, 25):
            #Rever o codigo para ver a necessidade de transpor a matriz
            bS[j] = np.sum(np.power((x - aS[:25,j]),6))

        op = np.power(1/500 + np.sum(1/(bS)),(-1))

        return op
    #Working
    def func13_op(self, x):

        aK = [.1957, .1947, .1735, .16, .0844, .0627, .0456, .0342, .0323, .0235, .0246]
        bK = [.25, .5, 1, 2, 4, 6, 8, 10, 12, 14, 16]
        
        for i in range(len(bK)):
            bK[i] = 1/bK[i]
            op = np.sum(np.power(aK[i]-((x * (np.power(bK[i],2)+x*bK[i]))/(np.power(bK[i],2)+x * bK[i]+x)),2))

        return op
    #Not working
    def func14_op(self, x):

        op = (1 + np.power((x + x+1),2) * (19-14*x+3*np.power(x,2)-14*x+6*x*x+3*np.power(x,2))) * (np.power(30+(2*x-3*x),2)*(18-32*x+12*np.power(x,2)+48*x-36*x*x+27*np.power(x,2)))

        return op
    #Working
    def func15_op(self, x):

        aH = [[3, 10, 30], [0.1, 10, 35], [3, 10, 30], [0.1, 10, 35]]
        cH = [1, 1.2, 3, 3.2]
        pH = [[0.3689, 0.117, 0.2673], [0.4699, 0.4387, 0.747], [0.1091, 0.8732, 0.5547], [0.03815, 0.5743, 0.8828]]
        op = 0

        for i in range(1,4):
            op = op - cH[i]*math.exp(-(np.sum(aH[i:]*(np.power(x-pH[i:],2)))))

        return op
    #Not working
    def func16_op(self, x):

        aH = [[10, 3, 17, 3.5, 1.7, 8], [0.05, 10, 17, 0.1, 8, 14], [3, 3.5, 1.7, 10, 17, 8], [17, 8, 0.05, 10, 0.1, 14]]
        cH = [1, 1.2, 3, 3.2]
        pH = [[0.1312, 0.1696, 0.5569, 0.0124, 0.8283, 0.5886], [0.2329, 0.4135, 0.8307, 0.3736, 0.1004, 0.9991], [0.2348, 0.1415, 0.3522, 0.2883, 0.3047, 0.6650], [0.4047, 0.8828, 0.8732, 0.5743, 0.1091, 0.0381]]
        op = 0

        for i in range(1,4):
            op = op - cH[i]*math.exp(-(np.sum(aH[i,:]* (np.power((x-pH[1,:]),2)))))

        return op
    #Not working
    def func17_op(self, x):

        aSH = [[4, 4, 4, 4], [1, 1, 1, 1], [8, 8, 8, 8], [6, 6, 6, 6], [3, 7, 3, 7], [2, 9, 2, 9], [5, 5, 3, 3], [8, 1, 8, 1], [6, 2, 6, 2], [7, 3.6, 7, 3.6]]
        cSH = [0.1, 0.2, 0.2, 0.4, 0.4, 0.6, 0.3, 0.7, 0.5, 0.5]
        op = 0

        for i in range(1,5):
            #Apostrofo
            op = op - np.power(((x-aSH[1:])*(x-aSH[i:])+cSH[i]),(-1))

        return op
    #Not working
    def func18_op(self, x):

        aSH = [[4, 4, 4, 4], [1, 1, 1, 1], [8, 8, 8, 8], [6, 6, 6, 6], [3, 7, 3, 7], [2, 9, 2, 9], [5, 5, 3, 3], [8, 1, 8, 1], [6, 2, 6, 2], [7, 3.6, 7, 3.6]]
        cSH = [0.1, 0.2, 0.2, 0.4, 0.4, 0.6, 0.3, 0.7, 0.5, 0.5]
        op = 0
        for i in range(1, 7):
            #Apostrofo
            op = op - np.power(((x - aSH[i,:])*(x-aSH[i,:])+cSH[i]),(-1))

        return op
    #Not working
    def func19_op(self, x):

        aSH = [[4, 4, 4, 4], [1, 1, 1, 1], [8, 8, 8, 8], [6, 6, 6, 6], [3, 7, 3, 7], [2, 9, 2, 9], [5, 5, 3, 3], [8, 1, 8, 1], [6, 2, 6, 2], [7, 3.6, 7, 3.6]]
        cSH = [0.1, 0.2, 0.2, 0.4, 0.4, 0.6, 0.3, 0.7, 0.5, 0.5]
        op = 0

        for i in range(1, 10):
            #Apostrofo
            op = op - np.power(((x-aSH[i,:])*(x-aSH[i,:]) + cSH[i]),(-1))

        return op


### Main code

In [41]:
#Create class
collie = functions()

#Population size
n = 30

#Maximum no. of iterations
gen = 200

#Optimization function name
lower_b, upper_b, dim = collie.func15()

#Intialize the population(init_p-Population,acc-acceleration of each individual)
init_p, acc = generate(n, dim, upper_b, lower_b)
#Vt = velocity of each individuals
Vt = np.zeros((n, dim))

#Time of each individual
time = np.random.random(size = (n,1))
#Max fitness value
fopt = math.inf
#Variable to store fitness
fit = np.zeros((n, 1))
pop = init_p
#k = counter variable for iterations  required for Eyeing mechanism
k = 1
fopt_1 = np.zeros((gen, 1))
for g in range(gen):
    #Calculate fitness of indivuals
    fit, maxf, pos = fitness(pop, n, dim, upper_b, lower_b, collie)
    
    eye = 0
    if g == 0:
        fopt = maxf
        
    #Finding the optimum fitness value
    if fopt > maxf:
        fopt = maxf
        
    fopt_1[g] = fopt
    if g > 0:
        if fopt_1[g] > fopt_1[g-1]:
            k += 1
            if k > 5:
                eye = 1
                k = 0
    
    pop, Vt, fit, acc, time = herding(pop, Vt, fit, n, dim, acc, time)
    #Funcionando
    Vt, acc, time, r1, l1, tempg, temps = update_v(Vt, n, dim, acc, time, pop, fit, eye)
    #Funcionando
    pop = update(pop, Vt, time, acc, n, dim, eye)
    #Funcionando
    pop, acc, time, Vt = check(pop, n, dim, upper_b, lower_b, acc, Vt, time)

    print("\n\n fopt  %.4f\t maxf %.4f\t gen %d"%(fopt, maxf, g))


C:\Users\artur\AppData\Local\Temp/ipykernel_14676/671976675.py:60: RuntimeWarning: invalid value encountered in double_scalars
  s += ((Vt1[i][j]) - (Vt[i][j])) / (acc[i][j])




 fopt  -2.4741	 maxf -2.4741	 gen 0


 fopt  -2.4741	 maxf -0.4432	 gen 1


 fopt  -2.6315	 maxf -2.6315	 gen 2


 fopt  -2.6315	 maxf -0.0645	 gen 3


 fopt  -2.6315	 maxf -0.3742	 gen 4


 fopt  -2.6315	 maxf -0.0239	 gen 5


 fopt  -2.6315	 maxf -2.5748	 gen 6


 fopt  -2.6315	 maxf -1.4054	 gen 7


 fopt  -2.9821	 maxf -2.9821	 gen 8


 fopt  -2.9821	 maxf -0.0047	 gen 9


 fopt  -2.9821	 maxf -2.5979	 gen 10


 fopt  -2.9821	 maxf -1.0426	 gen 11


 fopt  -2.9821	 maxf -0.1299	 gen 12


 fopt  -2.9821	 maxf -1.5258	 gen 13


 fopt  -2.9821	 maxf -0.1860	 gen 14


 fopt  -2.9821	 maxf -0.0427	 gen 15


 fopt  -2.9821	 maxf -0.6092	 gen 16


 fopt  -3.0130	 maxf -3.0130	 gen 17


 fopt  -3.0130	 maxf -1.3193	 gen 18


 fopt  -3.0130	 maxf -2.0439	 gen 19


 fopt  -3.0130	 maxf -2.1102	 gen 20


 fopt  -3.0130	 maxf -2.0008	 gen 21


 fopt  -3.0130	 maxf -0.3023	 gen 22


 fopt  -3.0130	 maxf -0.5749	 gen 23


 fopt  -3.0130	 maxf -0.0009	 gen 24


 fopt  -3.0130	 maxf -2.7589	 gen

: 